To run this file we just need to upload data.tsv file into a folder "data" and execute the code blocks in order.



In [22]:
import torch
import torchtext
from torchtext import data
import torch.optim as optim
import argparse
import os
import pandas as pd

In [23]:
import csv
count_positive = 0
count_negative = 0
tsv_file = open("/content/data/data.tsv")
read_tsv = csv. reader(tsv_file, delimiter="\t")
for row in read_tsv:
  if row[1] == '1':
    count_positive = count_positive+1
  elif row[1] == '0':
    count_negative = count_negative+1

print("Number of Positive examples:",count_positive)
print("Number of Negative examples:",count_negative)
tsv_file. close()

Number of Positive examples: 5000
Number of Negative examples: 5000


In [24]:
import pandas as pd
from sklearn.model_selection import cross_validate, train_test_split

df = pd.read_table('/content/data/data.tsv')

from sklearn.model_selection import train_test_split

print(df)

X = df.drop(columns = ['label']).copy()
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.20, random_state=1)

print(X_train.shape), print(y_train.shape)
print(X_val.shape), print(y_val.shape)
print(X_test.shape), print(y_test.shape)

print(len(X_train))
print(len(X_val))
print(len(X_test))


                                                   text  label
0     smart and alert , thirteen conversations about...      1
1     color , musical bounce and warm seas lapping o...      1
2     it is not a mass-market entertainment but an u...      1
3     a light-hearted french film about the spiritua...      1
4     my wife is an actress has its moments in looki...      1
...                                                 ...    ...
9995  in the end , they discover that balance in lif...      0
9996  a counterfeit 1000 tomin bank note is passed i...      0
9997  enter the beautiful and mysterious secret agen...      0
9998  after listening to a missionary from china spe...      0
9999  looking for a short cut to fame , glass concoc...      0

[10000 rows x 2 columns]
(6400, 1)
(6400,)
(1600, 1)
(1600,)
(2000, 1)
(2000,)
6400
1600
2000


In [25]:
type(X_train)

pandas.core.frame.DataFrame

In [26]:
df_train = pd.concat([X_train, y_train], axis=1)
df_train

,text,label
8979,"at the same time , mickey steals money from an...",0
1903,two hours of sepia-tinted heavy metal images a...,1
8191,"amongst these , ruth , a girl of 17 , writes a...",0
8276,"as an additional inducement , the queen holds ...",0
365,directed without the expected flair or imagina...,1
...,...,...
7586,matters are complicated still further when bec...,0
8288,"because of his shame , bobby undergoes a type ...",0
9718,young marrieds lori and tom relocate from chic...,0
7562,the story begins with the discovery that otte'...,0


In [27]:
s0 = df_train.label[df.label.eq(0)].sample(25).index
s1 = df_train.label[df.label.eq(1)].sample(25).index 

df_50 = df_train.loc[s0.union(s1)]

In [28]:
df_50.to_csv("/content/data/overfit.tsv", sep = "\t")

In [29]:
df_overfit = pd.read_table('/content/data/overfit.tsv')
print(df_overfit)

    Unnamed: 0                                               text  label
0          256  much as we might be interested in gratuitous s...      1
1          273  there is no substitute for on-screen chemistry...      1
2          349  it's simply delightful to watch an actor like ...      1
3          860  a comedy-drama of nearly epic proportions root...      1
4         1116  hey arnold ! the movie is what happens when yo...      1
5         1395  this is as respectful a film as byatt fans cou...      1
6         2004  two tedious acts light on great scares and a g...      1
7         2150  every single scene conveys a significant and c...      1
8         2156  a comedy that swings and jostles to the rhythm...      1
9         2295  gooding is the energetic frontman , and it's h...      1
10        2798  [shyamalan] turns the goose-pimple genre on it...      1
11        3328  chelsea walls is a case of too many chefs fuss...      1
12        3523  it's supposed to be a romantic come

In [30]:
def createDatasets(df):
    train, test = train_test_split(df[['text', 'label']], test_size=0.2, random_state=0, stratify=df[['label']])

    trainFinal, val = train_test_split(train, test_size=0.2, random_state=0, stratify=train['label'])

    #writing to train, val and test tsv
    trainFinal.to_csv('data/train.tsv', sep="\t", index=False)
    test.to_csv('data/test.tsv', sep="\t", index=False)
    val.to_csv('data/validation.tsv', sep="\t", index=False)

    #creating overfit dataset and writing to tsv
    s0 = trainFinal.label[df.label.eq(0)].sample(25).index
    s1 = trainFinal.label[df.label.eq(1)].sample(25).index 

    overfit = trainFinal.loc[s0.union(s1)]
    overfit.to_csv('data/overfit.tsv', sep="\t", index=False)

    #checking if number of positive and negative samples are the same
    if trainFinal[trainFinal.label == 1].shape[0] == trainFinal[trainFinal.label == 0].shape[0]:
        print("Number of positive and negative samples in the train set are equal.")
    if test[test.label == 1].shape[0] == test[test.label == 0].shape[0]:
        print("Number of positive and negative samples in the test set are equal.")
    if val[val.label == 1].shape[0] == val[val.label == 0].shape[0]:
        print("Number of positive and negative samples in the validation set are equal.")


    #checking if any common rows between train and test datasets
    df3  = pd.merge(trainFinal, test, on="text", how='inner', indicator='Exist')
    df3  = df3.loc[df3['Exist'] != 'both']
    print("Printing number of common rows between train and test datasets: ", df3.shape[0])

    if (df3.shape[0] == 0):
        print("Train and test datsets are unique with no common rows")


    #checking if any common rows between train and validation datasets
    df3  = pd.merge(trainFinal, val, on="text", how='inner', indicator='Exist')
    df3  = df3.loc[df3['Exist'] != 'both']
    print("Printing number of common rows between train and validation datasets: ", df3.shape[0])

    if (df3.shape[0] == 0):
        print("Train and validation datsets are unique with no common rows")


    #checking if any common rows between test and validation datasets
    df3  = pd.merge(test, val, on="text", how='inner', indicator='Exist')
    df3  = df3.loc[df3['Exist'] != 'both']
    print("Printing number of common rows between test and validation datasets: ", df3.shape[0])

    if (df3.shape[0] == 0):
        print("Test and validation datsets are unique with no common rows")

In [31]:
df = pd.read_csv("data/data.tsv", sep='\t')
createDatasets(df)

Number of positive and negative samples in the train set are equal.
Number of positive and negative samples in the test set are equal.
Number of positive and negative samples in the validation set are equal.
Printing number of common rows between train and test datasets:  0
Train and test datsets are unique with no common rows
Printing number of common rows between train and validation datasets:  0
Train and validation datsets are unique with no common rows
Printing number of common rows between test and validation datasets:  0
Test and validation datsets are unique with no common rows
